In [1]:
import serial
import time
import math
import subprocess
import os

In [2]:
# Serial port
stream = os.popen('powershell -command "[System.IO.Ports.SerialPort]::getportnames()"')
output = stream.read()[:-1] #return output of used usb, if no others are connected
output='COM3'

ser = serial.Serial(output,timeout=0.2) # Open serial port which is connected to the device
ser.baudrate = 57600
print("Serial port used: " + ser.name) # Check which port was really used

Serial port used: COM3


In [3]:
ser

Serial<id=0x22235c20460, open=True>(port='COM3', baudrate=57600, bytesize=8, parity='N', stopbits=1, timeout=5, xonxoff=False, rtscts=False, dsrdtr=False)

In [6]:
import binascii
binascii.unhexlify(b'128')

Error: Odd-length string

# Byte Analysis

In [14]:
###testing "wizard" sequence

#string='ÿ°Øð'
#a=string.encode()
#ser.write(b"\x1b\x00\x20\x3a\x8d\x57\x89\xc5\xff\xff\x00\x00\x00\x00\x09\x00\x01\x01\x00\x01\x00\x81\x03\x0d\x00\x00\x00\x01\x60\x0f\x11\x10\x05\x80\xa4\x02\x00\xf9\x45\xf0")
ser.write(b"\x0f\x10\xff\x05\xb0\x01\x02\x00\x11\xd8\xf0")
time.sleep(0.2)
ser.read_all()


b'\x0f\x11\x10\x03\xb0\x01\x00\xd5\xf0'

In [19]:
# LDEna - Laser driver enable
print("\nLDEna - Laser driver enable:")
b=b"\x0F\x11\xFF\x03\x40\x15\x00\x68\xF0"
#b=b"\x0F\x11\xFF\x03\x40\x16\x00\x69\xF0"
#this is the byte architecture
#Start of frame, Source Address, Destination Address, Length of data,[...Data...],Checksum, End of frame
#Data: Alarm-> CLDD temp. fatal threshold, System Register-> Laser Driver enable, No Op
ser.write(b)

time.sleep(0.5) # Wait 500 ms

i = 0
while(i < 9):
    i = i + 1
    byte = ser.read(1) # Read data byte
    print("byte (bin) = 0b" + "{0:08b}".format(ord(byte)))

time.sleep(3.000) # Wait 3 s


LDEna - Laser driver enable:
byte (bin) = 0b00001111
byte (bin) = 0b00010001
byte (bin) = 0b00010001
byte (bin) = 0b00000011
byte (bin) = 0b01000000
byte (bin) = 0b00010101
byte (bin) = 0b00000000
byte (bin) = 0b01111010
byte (bin) = 0b11110000


In [18]:
# DevType - Device type
print("\nDevType - Device type:")
b=b"\x0F\x11\xFF\x03\x80\x07\x00\x9A\xF0"
ser.write(b)

time.sleep(0.500) # Wait 500 ms

ser.read_all()
#i = 0
#while(i < 21):
   # i = i + 1
   # byte = ser.read(1) # Read data byte
   # print("byte (bin) = 0b" + "{0:08b}".format(ord(byte)))


DevType - Device type:


b"\x0f\x11\x11\x0f\x80\x07\x0cLASER DRIVER'\xf0"

In [9]:
# PLIPSet - Configures the laser diode current/power (write)
print("\nPLIPSet - Configures the laser diode current/power (write):")
#b=b"\x0f\x10\x11\x05\xb0\x72\x02\x00\xfb\x45\xf0"
def string_to_byte(x):
    if x<=197:
        h1=hex(x)[2:]
        h2=hex(145-(100-x))[2:]
        stringo="0F11FF07A0720400" + h1 + "0000" + h2 + 'F0'
        return bytes.fromhex(stringo)
    else:
        h1=hex(x)[2:]
        h2=hex(69-(251-x))[2:]
        stringo="0F101105B0720200" + h1 + h2 + 'F0'
        return bytes.fromhex(stringo)
    
#b=b"\x0F\x11\xFF\x07\xA0\x72\x04\x00\x32\x00\x00\x5F\xF0" #0x64=100

b=string_to_byte(50)
ser.write(b)
#this is the byte architecture
#Start of frame, Source Address, Destination Address, Length of data,[...Data...],Checksum, End of frame
#Data: Monitorization->CLDD temp.,Current/Power config, create lengthed buffer, buffer...
time.sleep(0.500) # Wait 500 ms

print(ser.read_all())

#i = 0
#while(i < 9):
 #   i = i + 1
  #  byte = ser.read(1) # Read data byte
   # print("byte (bin) = 0b" + "{0:08b}".format(ord(byte)))


PLIPSet - Configures the laser diode current/power (write):
b'\x0f\x11\x11\x03\xa0r\x007\xf0'


In [8]:
# PLIPSet - Configures the laser diode current/power (read)
print("\nPLIPSet - Configures the laser diode current/power (read):")
b=b"\x0F\x11\xFF\x03\x80\xA1\x00\x34\xF0" #current
#data: laser output power, config diode current/power, No op
ser.write(b)

time.sleep(0.500) # Wait 500 ms

a=ser.read_all()
print(a)
print(a[-3])
#i = 0
#while(i < 11):
 #   i = i + 1
  #  byte = ser.read(1) # Read data byte
   # if (i==9 or i==10):
    #    print(int(byte[1:], base=16),byte)


PLIPSet - Configures the laser diode current/power (read):
b'\x0f\x11\x11\x03D\x13\x00|\xf0\x0f\x11\x11\x03@\x14\x00y\xf0\x0f\x11\x11\x05\x80\xa1\x02\x00\x00J\xf0'
0


In [17]:
# TECTSet - Configures the TEC temperature (write)
print("\nTECTSet - Configures the TEC temperature (write):")
#b=b"\x0F\x11\xFF\x06\xA0\x75\x03\x00\xC8\x00\xF6\xF0"
#b=b"\x0f\x10\x11\x05\xb0\x75\x02\x00\xfb\x48\xf0"

def string_to_byte1(x):
    h1=hex(x)[2:]
    h2=hex(72-(251-x))[2:]
    print(h1,h2)
    stringo="0F101105B0750200" + h1 + h2 + 'F0'
    print(stringo)
    return bytes.fromhex(stringo)

#C8=200
b=string_to_byte1(250) #ºC*10
ser.write(b)
time.sleep(0.500) # Wait 500 ms

ser.read_all()
#i = 0
#while(i < 9):
#    i = i + 1
 #   byte = ser.read(1) # Read data byte
  #  print("byte (bin) = 0b" + "{0:08b}".format(ord(byte)))


TECTSet - Configures the TEC temperature (write):
fa 47
0F101105B0750200fa47F0


b'\x0f\x11\x10\x03\xb0u\x00I\xf0'

In [26]:
# TECTSet - Configures the TEC temperature (read)
print("\nTECTSet - Configures the TEC temperature (read):")
b=b"\x0F\x11\xFF\x03\x80\x75\x00\x08\xF0"
ser.write(b)

time.sleep(0.500) # Wait 500 ms

a=ser.read_all()
print(a[-3])
print(a[-2])

#i = 0
#while(i < 11):
 #   i = i + 1
  #  byte = ser.read(1) # Read data byte
     #print("byte (bin) = 0b" + "{0:08b}".format(ord(byte)))
b=b"\x0F\x11\xFF\x03\x80\xA4\x00\x37\xF0" #this is the real controller
ser.write(b)

time.sleep(0.08) # Wait 500 ms

a=ser.read_all()
print(a[-3])
print(a[-2])


TECTSet - Configures the TEC temperature (read):


NameError: name 'ser' is not defined

In [32]:
# LDDis - Laser driver disable
print("\nLDDis - Laser driver disable:")
b=b"\x0F\x11\xFF\x03\x40\x16\x00\x69\xF0"
#b=b"\x0F\x11\xFF\x03\x40\x15\x00\x68\xF0"
ser.write(b)

time.sleep(0.500) # Wait 500 ms

#print(ser.read_all())
i = 0
while(i < 9):
    i = i + 1
    byte = ser.read(1) # Read data byte
    print("byte (bin) = 0b" + "{0:08b}".format(ord(byte)))


LDDis - Laser driver disable:
byte (bin) = 0b00001111
byte (bin) = 0b00010001
byte (bin) = 0b00010001
byte (bin) = 0b00000011
byte (bin) = 0b01000000
byte (bin) = 0b00010110
byte (bin) = 0b00000000
byte (bin) = 0b01111011
byte (bin) = 0b11110000


In [4]:
#restart
b=b"\x0F\x11\xFF\x03\x80\xA4\x00\x37\xF0"
ser.write(b)
#time.sleep(0.05)
a=ser.read_until()
print(a)
print(a[-3])
print(a[-2])
print(7600*(a[-3]*(2.5/2**10)-1.25))

b'\x0f\x11\x11\x05\x80\xa4\x02\x00\x00M\xf0'
0
77
-9500.0


In [41]:
#b=b"\x0F\x11\xFF\x06\xA0\x75\x03\x00\xC8\x00\xF6\xF0"
#b=b"\x0f\x10\x11\x05\xa0\x25\x02\x00\x00\xed\xf0"
#b=b"\x0f\x10\x11\x05\xb0\x75\x02\x00\xfa\x47\xf0"
#b=b"\x0f\x10\x11\x05\x80\x10\x02\x00\x00\xb8\xf0"
#b=b"\x0f\x10\x11\x05\xb0\x72\x02\x00\xfb\x45\xf0"
b=b"\x0f\x10\x11\x05\x80\x10\x02\x00\x00\xb8\xf0"
ser.write(b)
#time.sleep(0.2)
statuse=''
a=ser.read_all()
print(a)
dig= a[-3] if len(str(a[-2]))>len(str(a[-3])) else a[-2]
if dig in [36,66,28]:
    statuse='Disabled'
if dig in [7,71]:
    statuse='Hibernating'
if dig in [5,53,37,117,45]:
    statuse='Active'
if dig in [2]:
    statuse='Ready'
if dig in [67,69,29]:
    statuse='Configuring'
else:
    pass
        
print(statuse)

b'\x0f\x11\x10\x05\x80\x10\x02\x00d\x1c\xf0'
Disabled


In [42]:
# Serial port
ser.close() # Close serial port
print("\nSerial port closed")


Serial port closed


# Classe

In [1]:
import serial
import time
import os

stream = os.popen('powershell -command "[System.IO.Ports.SerialPort]::getportnames()"')
output = stream.read()[:-1] #return output of used usb, if no others are connected
output='COM4'

class mwLaser():
    
    def __init__(self):
        super().__init__()
        
        self.port = serial.Serial(output) # Open serial port which is connected to the device
        self.port.baudrate = 57600
        self.current = 0
        self.power = 0
        self.TEC_temperature = 0
        self.TEC_current=0
        self.status=''
        self.laser_status()
        
    def laser_status(self):
        time.sleep(0.3)
        b=b"\x0f\x10\x11\x05\x80\x10\x02\x00\x00\xb8\xf0"
        self.port.write(b)
        time.sleep(0.09)
        statuse=''
        a=self.port.read_all()
        print(a)
        dig= a[-3] if len(str(a[-2]))>len(str(a[-3])) else a[-2]
        if dig in [36,66,28]:
            statuse='Disabled'
        if dig in [7,71]:
            statuse='Hibernating'
        if dig in [5,53,37,117,45]:
            statuse='Active'
        if dig in [2]:
            statuse='Ready'
        if dig in [67,69,29]:
            statuse='Configuring'
        
        self.status=statuse
        
    def update_current(self):
        b=b"\x0F\x11\xFF\x03\x80\xA1\x00\x34\xF0"
        self.port.write(b)
        time.sleep(0.15)
        self.current=self.port.read_all()[-3]
        
    def update_power(self):
        b=b"\x0F\x11\xFF\x03\x80\xA2\x00\x35\xF0"
        self.port.write(b)
        time.sleep(0.15)
        self.power=self.port.read_all()[-3]
    
    def update_TEC_temperature(self):
        b=b"\x0F\x11\xFF\x03\x80\xA4\x00\x37\xF0"
        self.port.write(b)
        time.sleep(0.15)
        self.TEC_temperature=self.port.read_all()[-3]/10
    
    def update_TEC_current(self):
        b=b"\x0F\x11\xFF\x03\x80\xA5\x00\x38\xF0"
        self.port.write(b)
        time.sleep(0.15)
        self.TEC_current=7600*(self.port.read_all()[-3]*(2.5/2**10)-1.25)/100
    
    def set_TEC_temperature(self,x):
        h1=hex(x)[2:]
        h2=hex(246-(200-x))[2:]
        stringo="0F101105B0750200" + h1 + h2 + 'F0'
        b=bytes.fromhex(stringo)
        self.port.write(b)
        time.sleep(0.09)
        
    def set_current(self,x):
        if x<=197:
            h1=hex(x)[2:]
            h2=hex(145-(100-x))[2:]
            stringo="0F11FF07A0720400" + h1 + "0000" + h2 + 'F0'
            b=bytes.fromhex(stringo)
            self.port.write(b)
            time.sleep(0.09)
        else:
            h1=hex(x)[2:]
            h2=hex(69-(251-x))[2:]
            stringo="0F101105B0720200" + h1 + h2 + 'F0'
            b=bytes.fromhex(stringo)
            self.port.write(b)
            time.sleep(0.09)
    
    def return_data(self):
        self.update_current()
        self.update_power()
        self.update_TEC_temperature()
        self.update_TEC_current()
        self.laser_status()
        return [self.current,self.power,self.TEC_temperature,self.TEC_current,self.status]
    
    def enable(self):
        b=b"\x0F\x11\xFF\x03\x40\x15\x00\x68\xF0"
        self.port.write(b)
        
        
    def disable(self):
        b=b"\x0F\x11\xFF\x03\x40\x16\x00\x69\xF0"
        self.port.write(b)
    
    def reset(self):
        time.sleep(0.09)
        b=b"\x0F\x11\xFF\x03\x40\x14\x00\x67\xF0"
        self.port.write(b)
        
    def hibernate(self):
        self.disable()
        time.sleep(0.09)
        b=b"\x0F\x11\xFF\x03\x40\x13\x00\x66\xF0"
        self.port.write(b)
    
    def close_port(self):
        self.port.close()
        print("\nSerial port closed")

In [2]:
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
#import breeze_resources
import time

import sys

class laser_widget(QWidget):
    """
    This is my widget.
    """
    def __init__(self, parent=None):
        
        super().__init__()
        
        self.global_layout = QGridLayout(self)
        
        self.laser=mwLaser() ##laser driver
        self.status=self.laser.status
        #######################################################laser box
        container_laser=QGroupBox("Laser")
        vbox_laser= QVBoxLayout(container_laser)
        
        containersimple=QGroupBox()
        hboxsimple=QHBoxLayout(containersimple)
        self.current_display=QLCDNumber(self)
        self.current_display.setGeometry(QRect())
        hboxsimple.addWidget(self.current_display)
        self.labelmAl=QLabel(self)
        self.labelmAl.setText('mA')
        hboxsimple.addWidget(self.labelmAl)
        vbox_laser.addWidget(containersimple)
        
        containersimple1=QGroupBox()
        hboxsimple1=QHBoxLayout(containersimple1)
        self.power_display=QLCDNumber(self)
        self.power_display.setGeometry(QRect())
        hboxsimple1.addWidget(self.power_display)
        self.labelmW=QLabel(self)
        self.labelmW.setText('mW')
        hboxsimple1.addWidget(self.labelmW)
        hboxsimple1.addWidget(containersimple1)
        vbox_laser.addWidget(containersimple1)
        
        ################################################TEC box
        container_TEC=QGroupBox("TEC")
        vbox_TEC= QVBoxLayout(container_TEC)
        containersimple2=QGroupBox()
        
        hboxsimple2=QHBoxLayout(containersimple2)
        self.temp_display=QLCDNumber(self)
        self.temp_display.setGeometry(QRect())
        hboxsimple2.addWidget(self.temp_display)
        self.labelC=QLabel(self)
        self.labelC.setText('ºC')
        hboxsimple2.addWidget(self.labelC)
        vbox_TEC.addWidget(containersimple2)
        
        containersimple3=QGroupBox()
        hboxsimple3=QHBoxLayout(containersimple3)
        self.currTEC_display=QLCDNumber(self)
        self.currTEC_display.setGeometry(QRect())
        hboxsimple3.addWidget(self.currTEC_display)
        self.labelmATEC=QLabel(self)
        self.labelmATEC.setText('mA')
        hboxsimple3.addWidget(self.labelmATEC)
        vbox_TEC.addWidget(containersimple3)
        
        ###############################################current control
        container_input=QGroupBox("Laser Setpoint")
        hbox_set= QHBoxLayout(container_input)
        
        self.labelcur=QLabel(self)
        self.labelcur.setText('Current setpoint:')
        hbox_set.addWidget(self.labelcur)
        
        self.currentv=QLineEdit(self)
        self.currentv.setText('50')
        self.currentv.returnPressed.connect(self.current_changed)
        hbox_set.addWidget(self.currentv)
        
        self.labelmA=QLabel(self)
        self.labelmA.setText('mA')
        hbox_set.addWidget(self.labelmA)
        
        ##################################################temp control
        container_inputt=QGroupBox("TEC Setpoint")
        hbox_set1= QHBoxLayout(container_inputt)
        
        self.labeltemps=QLabel(self)
        self.labeltemps.setText('Temperature setpoint:')
        hbox_set1.addWidget(self.labeltemps)
        
        self.temperaturev=QLineEdit(self)
        self.temperaturev.setText('25')
        self.temperaturev.returnPressed.connect(self.temp_changed)
        hbox_set1.addWidget(self.temperaturev)
        
        self.labelC=QLabel(self)
        self.labelC.setText('ºC')
        hbox_set1.addWidget(self.labelC)
        
        ########################################################buttons
        container_button=QGroupBox()
        hbox_set2= QHBoxLayout(container_button)
        
        self.turn_on_button=QToolButton(self)
        self.turn_on_button.setText('Start Laser')
        self.turn_on_button.clicked.connect(self.laser_clicked)
        self.turn_on_button.clicked.connect(self.button_state)
        
        container_button2=QGroupBox()
        hbox_set3= QHBoxLayout(container_button2)
        
        self.shutdown_button=QToolButton(self)
        self.shutdown_button.setText('Shutdown')
        self.shutdown_button.clicked.connect(self.shutdown)
        self.shutdown_button.clicked.connect(self.button_state)
        
        hbox_set2.addWidget(self.turn_on_button)
        hbox_set3.addWidget(self.shutdown_button)

        ######################################################global
        
        self.global_layout.addWidget(container_laser,0,0)
        self.global_layout.addWidget(container_TEC,0,1)
        self.global_layout.addWidget(container_input,1,0)
        self.global_layout.addWidget(container_inputt,1,1)
        self.global_layout.addWidget(container_button,2,1)
        self.global_layout.addWidget(container_button2,2,0)
        
        #self.setFixedWidth(100)
        #self.setFixedHeight(200)
    ###########################################################functions
    #### initial state   
        self.thread()
        self.button_state()
    #### initiate thread and change status
    
    def thread(self):
        self.thread=QThread()
        self.worker=Worker(self.laser)
        self.worker.moveToThread(self.thread)
        self.thread.started.connect(self.worker.update)
        self.worker.finished.connect(self.thread.quit)
        self.worker.finished.connect(self.worker.deleteLater)
        self.thread.finished.connect(self.thread.deleteLater)
        self.worker.data.connect(self.update_status)
        self.thread.start()
    
    
    #### get status from laser class
    def update_status(self,data_retrieved):
        self.laser.current = data_retrieved[0]
        self.laser.power = data_retrieved[1]
        self.laser.TEC_temperature = data_retrieved[2]
        self.laser.TEC_curent=data_retrieved[3]
        self.laser.status=data_retrieved[4]
        self.update_laser_values()
    
    #### 
    def update_laser_values(self):
        self.current_display.display(self.laser.current)
        self.power_display.display(self.laser.power)
        self.currTEC_display.display(self.laser.TEC_current)
        self.temp_display.display(self.laser.TEC_temperature)
        self.status=self.laser.status
        self.button_state()
        
    ##button methods
    def button_state(self):
        if self.status=='Disabled':
            self.turn_on_button.setText('Start Laser')
        
        elif self.status=='Active':
            self.turn_on_button.setText('Disable')

        elif self.status=='Hibernating':
            self.turn_on_button.setText('Restart')
        
        elif self.status=='Ready':
            self.turn_on_button.setText('Start Laser')
    
        elif self.status=='Configuring':
            self.turn_on_button.setText('Disable')
            
            
    def laser_clicked(self):
        if self.status=='Disabled':
            self.laser.enable()
            time.sleep(1) ## 2 seconds works
            
        elif self.status=='Active':
            self.laser.disable()
            time.sleep(1)
            
        elif self.status=='Hibernating':
            self.laser.reset()
            time.sleep(1)
        
        elif self.status=='Ready':
            self.laser.enable()
            time.sleep(1)
    
        elif self.status=='Configuring':
            self.laser.disable()
            time.sleep(1)
    
    def shutdown(self):
        self.laser.hibernate()
        time.sleep(1)
        
    def current_changed(self):
        print(self.currentv.text())
        self.laser.set_current(int(self.currentv.text()))
    
    def temp_changed(self):
        print(self.temperaturev.text())
        self.laser.set_TEC_temperature(int(self.temperaturev.text())*10)
        
    
    ##############################################laser thread
        
    def temp_alarm(self):
        if (float(self.laser.TEC_temperature)<24.9 or (self.laser.TEC_temperature)>25.1):
            msg = QMessageBox()
            msg.setIcon(QMessageBox.Critical)
            msg.setText("Error")
            msg.setInformativeText('TEC Temperature is unstable, shutting down the laser')
            msg.setWindowTitle("Error")
            msg.exec_() 
    
    def closeEvent(self, event):
        self.worker.stop()
        self.thread.quit()
        self.laser.close_port()
        event.accept()
        
class Worker(QObject):
    finished= pyqtSignal()
    data=pyqtSignal(list)
    
    def __init__(self,laser):
        super(Worker,self).__init__()
        self.laser=laser
        self.flag=True
        
    def update(self):
        while self.flag:
            data=self.laser.return_data()
            self.data.emit(data)
        self.finished.emit()
    
    def start(self):
        self.flag=True
        
    def stop(self):
        self.flag=False
    
qapp = QCoreApplication.instance()
if qapp is None:
    qapp = QApplication(sys.argv)
#     file = QFile(":/dark/stylesheet.qss")
#     file.open(QFile.ReadOnly | QFile.Text)
#     stream = QTextStream(file)
#     qapp.setStyleSheet(stream.readAll())
    
if __name__ == "__main__": 
    #start the widget
    ui = laser_widget()
    #show the widget
    ui.show()
    #start the events loop
    qapp.exec_()

b'\x0f\x11\x10\x05\x80\x10\x02\x005\xed\xf0'
b'\x0f\x11\x10\x05\x80\x10\x02\x005\xed\xf0'
b'\x0f\x11\x10\x05\x80\x10\x02\x005\xed\xf0'


IndexError: index out of range


Serial port closed


# Queueing Experiments

In [12]:
import threading, queue
import numpy as np

q = queue.Queue()

def worker():
    while True:
        item = q.get()
        print(f'Working on {item}')
        print(f'Finished {item}')
        q.task_done()
        

# Turn-on the worker thread.
threading.Thread(target=worker, daemon=True).start()

# Send thirty task requests to the worker.
for item in range(n):
    q.put() # only accepts data types, not functions

# Block until all tasks are done.
q.join()
print('All work completed')

Working on False
Finished False
[1.01496496e-311 1.02271589e-321 0.00000000e+000 0.00000000e+000
 0.00000000e+000 5.02034658e+175 1.70382599e-051 2.25160040e+184
 1.28992489e+165 1.17452773e+165 6.44385836e-067 4.26216559e-096
 1.12855837e+277 8.76730979e+252 2.59027920e-144 3.80985069e+180
 1.14428494e+243 2.59027907e-144 7.79952704e-143 2.23344829e-052
 2.83032493e-056 3.60360390e+175 8.24278336e-071 2.59027905e-144
 2.59903818e-144 6.19410686e-091 6.76090883e-067 6.52515944e-038
 1.11475752e+261 1.16318408e-028]
Working on False
Finished False
[1.01496496e-311 1.02271589e-321 0.00000000e+000 0.00000000e+000
 0.00000000e+000 5.02034658e+175 1.70382599e-051 2.25160040e+184
 1.28992489e+165 1.17452773e+165 6.44385836e-067 4.26216559e-096
 1.12855837e+277 8.76730979e+252 2.59027920e-144 3.80985069e+180
 1.14428494e+243 2.59027907e-144 7.79952704e-143 2.23344829e-052
 2.83032493e-056 3.60360390e+175 8.24278336e-071 2.59027905e-144
 2.59903818e-144 6.19410686e-091 6.76090883e-067 6.525159

# Queue Implementation 

In [1]:
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
import time
import queue
from laser_utils.send_requests import req_Worker
from laser_utils.event_handler import Worker
from laser_utils.laser_Copy1 import mwLaser
#import breeze_resources

import sys

class laser_widget(QWidget):
    """
    This is my widget.
    """
    def __init__(self, parent=None):
        
        super().__init__()
        
        self.global_layout = QGridLayout(self)
        
        self.laser=mwLaser() ##laser driver
        
        self.q = queue.Queue(maxsize=10) ## Event queue
        
        #######################################################laser box
        container_laser=QGroupBox("Laser")
        vbox_laser= QVBoxLayout(container_laser)
        
        containersimple=QGroupBox()
        hboxsimple=QHBoxLayout(containersimple)
        self.current_display=QLCDNumber(self)
        self.current_display.setGeometry(QRect())
        hboxsimple.addWidget(self.current_display)
        self.labelmAl=QLabel(self)
        self.labelmAl.setText('mA')
        hboxsimple.addWidget(self.labelmAl)
        vbox_laser.addWidget(containersimple)
        
        containersimple1=QGroupBox()
        hboxsimple1=QHBoxLayout(containersimple1)
        self.power_display=QLCDNumber(self)
        self.power_display.setGeometry(QRect())
        hboxsimple1.addWidget(self.power_display)
        self.labelmW=QLabel(self)
        self.labelmW.setText('mW')
        hboxsimple1.addWidget(self.labelmW)
        hboxsimple1.addWidget(containersimple1)
        vbox_laser.addWidget(containersimple1)
        
        ################################################TEC box
        container_TEC=QGroupBox("TEC")
        vbox_TEC= QVBoxLayout(container_TEC)
        containersimple2=QGroupBox()
        
        hboxsimple2=QHBoxLayout(containersimple2)
        self.temp_display=QLCDNumber(self)
        self.temp_display.setGeometry(QRect())
        hboxsimple2.addWidget(self.temp_display)
        self.labelC=QLabel(self)
        self.labelC.setText('ºC')
        hboxsimple2.addWidget(self.labelC)
        vbox_TEC.addWidget(containersimple2)
        
        containersimple3=QGroupBox()
        hboxsimple3=QHBoxLayout(containersimple3)
        self.currTEC_display=QLCDNumber(self)
        self.currTEC_display.setGeometry(QRect())
        hboxsimple3.addWidget(self.currTEC_display)
        self.labelmATEC=QLabel(self)
        self.labelmATEC.setText('mA')
        hboxsimple3.addWidget(self.labelmATEC)
        vbox_TEC.addWidget(containersimple3)
        
        ###############################################current control
        container_input=QGroupBox("Laser Setpoint")
        hbox_set= QHBoxLayout(container_input)
        
        self.labelcur=QLabel(self)
        self.labelcur.setText('Current setpoint:')
        hbox_set.addWidget(self.labelcur)
        
        self.currentv=QLineEdit(self)
        self.currentv.setText('50')
        self.currentv.returnPressed.connect(self.current_changed)
        hbox_set.addWidget(self.currentv)
        
        self.labelmA=QLabel(self)
        self.labelmA.setText('mA')
        hbox_set.addWidget(self.labelmA)
        
        ##################################################temp control
        container_inputt=QGroupBox("TEC Setpoint")
        hbox_set1= QHBoxLayout(container_inputt)
        
        self.labeltemps=QLabel(self)
        self.labeltemps.setText('Temperature setpoint:')
        hbox_set1.addWidget(self.labeltemps)
        
        self.temperaturev=QLineEdit(self)
        self.temperaturev.setText('25')
        self.temperaturev.returnPressed.connect(self.temp_changed)
        hbox_set1.addWidget(self.temperaturev)
        
        self.labelC=QLabel(self)
        self.labelC.setText('ºC')
        hbox_set1.addWidget(self.labelC)
        
        ########################################################buttons
        container_button=QGroupBox()
        hbox_set2= QHBoxLayout(container_button)
        
        self.turn_on_button=QToolButton(self)
        self.turn_on_button.setText('Start Laser')
        self.turn_on_button.clicked.connect(self.laser_clicked)
        #self.turn_on_button.clicked.connect(self.button_state)
        
        container_button2=QGroupBox()
        hbox_set3= QHBoxLayout(container_button2)
        
        self.shutdown_button=QToolButton(self)
        self.shutdown_button.setText('Shutdown')
        self.shutdown_button.clicked.connect(self.shutdown)
        #self.shutdown_button.clicked.connect(self.button_state)
        
        hbox_set2.addWidget(self.turn_on_button)
        hbox_set3.addWidget(self.shutdown_button)

        ######################################################global
        
        self.global_layout.addWidget(container_laser,0,0)
        self.global_layout.addWidget(container_TEC,0,1)
        self.global_layout.addWidget(container_input,1,0)
        self.global_layout.addWidget(container_inputt,1,1)
        self.global_layout.addWidget(container_button,2,1)
        self.global_layout.addWidget(container_button2,2,0)
        
        #self.setFixedWidth(100)
        #self.setFixedHeight(200)
    ###########################################################functions
    #### initial state   
        self.request_thread()
        self.thread()
        #self.button_state()
    #### initiate thread and change status
    
    def thread(self):
        self.thread=QThread()
        self.worker=Worker(self.q,self.laser)
        self.worker.moveToThread(self.thread)
        self.thread.started.connect(self.worker.update)
        self.worker.finished.connect(self.thread.quit)
        self.worker.finished.connect(self.worker.deleteLater)
        self.thread.finished.connect(self.thread.deleteLater)
        self.worker.data.connect(self.update_status)
        self.thread.start()
    
    def request_thread(self):
        self.reqthread=QThread()
        self.reqworker=req_Worker(self.q)
        self.reqworker.moveToThread(self.reqthread)
        self.reqthread.started.connect(self.reqworker.ask)
        self.reqworker.finished.connect(self.reqthread.quit)
        self.reqworker.finished.connect(self.reqworker.deleteLater)
        self.reqthread.finished.connect(self.reqthread.deleteLater)
        #self.reqworker.data.connect(self.results)
        self.reqthread.start()
    
    
    #### get status from laser class
    def update_status(self,data_retrieved):
        self.laser.current = data_retrieved[0]
        self.laser.power = data_retrieved[1]
        self.laser.TEC_temperature = data_retrieved[2]
        self.laser.TEC_curent=data_retrieved[3]
        self.laser.status=data_retrieved[4]
        self.update_laser_values()
    
    #### 
    def update_laser_values(self):
        self.current_display.display(self.laser.current)
        self.power_display.display(self.laser.power)
        self.currTEC_display.display(self.laser.TEC_current)
        self.temp_display.display(self.laser.TEC_temperature)
        self.status=self.laser.status
        self.button_state()
        
    def button_state(self):
        if self.status=='Disabled' or self.status=='Ready':
            self.turn_on_button.setText('Start Laser')
             
        elif self.status=='Active':
            self.turn_on_button.setText('Disable')

        elif self.status=='Hibernating':
            self.turn_on_button.setText('Restart')
    
        elif self.status=='Configuring':
            self.turn_on_button.setText('Disable')
    
    def shutdown(self):
        self.q.put_nowait('shutdown')
    
    def laser_clicked(self):
        if self.status=='Disabled':
            self.q.put_nowait('enable')
            
        elif self.status=='Active':
            self.q.put_nowait('disable')
            
        elif self.status=='Hibernating':
            self.q.put_nowait('reset')
        
        elif self.status=='Ready':
            self.q.put_nowait('enable')
    
        elif self.status=='Configuring':
            self.q.put_nowait('disable')
        
    def current_changed(self):
        print(self.currentv.text())
        self.laser.set_current(int(self.currentv.text()))
    
    def temp_changed(self):
        print(self.temperaturev.text())
        self.laser.set_TEC_temperature(int(self.temperaturev.text())*10)
        
    
    ##############################################laser thread
        
    def temp_alarm(self):
        if (float(self.laser.TEC_temperature)<24.9 or (self.laser.TEC_temperature)>25.1):
            msg = QMessageBox()
            msg.setIcon(QMessageBox.Critical)
            msg.setText("Error")
            msg.setInformativeText('TEC Temperature is unstable, shutting down the laser')
            msg.setWindowTitle("Error")
            msg.exec_() 
    
    def closeEvent(self, event):
        self.worker.stop()
        self.thread.quit()
        self.reqworker.stop()
        self.reqthread.quit()
        #self.q.join()
        print('Thread Closed')
        self.laser.close_port()
        event.accept()
    
        
    
qapp = QCoreApplication.instance()
if qapp is None:
    qapp = QApplication(sys.argv)
#     file = QFile(":/dark/stylesheet.qss")
#     file.open(QFile.ReadOnly | QFile.Text)
#     stream = QTextStream(file)
#     qapp.setStyleSheet(stream.readAll())
    
if __name__ == "__main__": 
    #start the widget
    ui = laser_widget()
    #show the widget
    ui.show()
    #start the events loop
    qapp.exec_()

['Sent request']
Task Done for update 1.4698479000000049
Queue length: 0
['Sent request']
Task Done for update 1.0124049
Queue length: 0
['Sent request']
Task Done for update 1.018361800000001
Queue length: 0
['Sent request']
Task Done for update 0.9286150000000006
Queue length: 0
['Sent request']
Task Done for update 1.0118916000000056
Queue length: 0
['Sent request']
Task Done for update 1.0181578000000044
Queue length: 0
['Sent request']
Task Done for update 1.0198912999999976
Queue length: 0
['Sent request']
Task Done for update 1.0272509999999997
Queue length: 0
['Sent request']
Task Done for update 1.0152155000000036
Queue length: 0
['Sent request']
Task Done for update 1.0181667000000019
Queue length: 0
['Sent request']
Task Done for update 1.0149072000000032
Queue length: 0
['Sent request']
Task Done for update 1.016205499999998
Queue length: 0
['Sent request']
Task Done for update 0.9301984000000019
Queue length: 0
['Sent request']
Task Done for update 1.0189057999999989
Queue

AttributeError: 'NoneType' object has no attribute 'hEvent'